In [1]:
import pandas as pd
import plotly.express as px


In [2]:

df_raw = pd.read_json(r'C:\Users\breno\Documents\Programacao\Projetos\notion-analytics\src\data\parsed\parsed_analytics_filtered_True.json')
df_raw.head(3)

,id,url,page_title,alternative_title,ds,unique_views,total_views
0,30c0c3ff-265e-4d09-9e58-f21361e5200d,https://www.notion.so/Instituto-Reditus-30c0c3...,Instituto Reditus,Instituto Reditus,2022-08-02,5,6
1,30c0c3ff-265e-4d09-9e58-f21361e5200d,https://www.notion.so/Instituto-Reditus-30c0c3...,Instituto Reditus,Instituto Reditus,2022-08-03,3,6
2,30c0c3ff-265e-4d09-9e58-f21361e5200d,https://www.notion.so/Instituto-Reditus-30c0c3...,Instituto Reditus,Instituto Reditus,2022-08-04,1,1


In [3]:
df = (
    df_raw
    .assign(date_visited=pd.to_datetime(df_raw['ds']))
    .sort_values(by='date_visited')
)
df.head(3)

,id,url,page_title,alternative_title,ds,unique_views,total_views,date_visited
0,30c0c3ff-265e-4d09-9e58-f21361e5200d,https://www.notion.so/Instituto-Reditus-30c0c3...,Instituto Reditus,Instituto Reditus,2022-08-02,5,6,2022-08-02
1,30c0c3ff-265e-4d09-9e58-f21361e5200d,https://www.notion.so/Instituto-Reditus-30c0c3...,Instituto Reditus,Instituto Reditus,2022-08-03,3,6,2022-08-03
847,dce4edc0-3ff6-4a2e-80e9-9bc0e4f4d6ac,https://www.notion.so/Miss-o-Vis-o-e-Valores-d...,"Missão, Visão e Valores","Missão, Visão e Valores",2022-08-03,1,1,2022-08-03


In [4]:
df.dtypes


id                           object
url                          object
page_title                   object
alternative_title            object
ds                           object
unique_views                  int64
total_views                   int64
date_visited         datetime64[ns]
dtype: object

In [11]:
# Aggregate the data by month
df['month_visited'] = df['date_visited'].dt.to_period('M').astype(str)
df_monthly = df.groupby(['month_visited', 'page_title']).agg({
    'unique_views': 'sum',
    'total_views': 'sum'
}).reset_index()

df_monthly.head()

,month_visited,page_title,unique_views,total_views
0,2022-08,Alterações do site,6,9
1,2022-08,Comunicação,8,13
2,2022-08,Dashboards,2,6
3,2022-08,Ferramentas,1,1
4,2022-08,Gestão,1,2


In [13]:
legend_order = df_monthly.groupby('page_title')['unique_views'].sum().sort_values(ascending=False).index.tolist()
fig_unique_monthly = px.line(df_monthly, x='month_visited', y='unique_views', color='page_title', title='Visualizações Únicas por Mês', category_orders={"page_title": legend_order})
fig_unique_monthly

In [14]:
# Reordering the legend based on the sum of 'total_views' for each 'page_title'
legend_order_total_views = df_monthly.groupby('page_title')['total_views'].sum().sort_values(ascending=False).index.tolist()

# Creating the line plot with custom legend order based on 'total_views'
fig_total_monthly = px.line(df_monthly, x='month_visited', y='total_views', color='page_title', 
                            title='Visualizações Totais por Mês',
                            category_orders={"page_title": legend_order_total_views})

# Show the figure
fig_total_monthly.show()


In [21]:
fig_total_monthly = px.line(df, x='date_visited', y='unique_views', color='page_title', 
                            title='Visualizações Totais por Mês')
fig_total_monthly.show()